<a href="https://colab.research.google.com/github/trista-paul/DS-Unit-4-Sprint-1-Tree-Ensembles/blob/master/Unit_4_Week_1_Lending_Club.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

!kaggle competitions download -c ds2-tree-ensembles

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/313k [00:00<?, ?B/s]
100% 313k/313k [00:00<00:00, 48.0MB/s]
  0% 0.00/3.57M [00:00<?, ?B/s]
100% 3.57M/3.57M [00:00<00:00, 117MB/s]
 99% 172M/174M [00:02<00:00, 75.9MB/s]
100% 174M/174M [00:02<00:00, 75.2MB/s]
100% 4.86M/4.86M [00:00<00:00, 39.4MB/s]



In [2]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip

Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       
unzip:  cannot find or open sample_submission.csv.zip, sample_submission.csv.zip.zip or sample_submission.csv.zip.ZIP.


In [7]:
!pip install category_encoders
!pip install dtreeviz

    100% |████████████████████████████████| 61kB 3.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/4c/3f/87/b2c828617d1bd181145fbaf41129e4c82fa8d9a98082701e00
Successfully built dtreeviz


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from dtreeviz.trees import dtreeviz
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score, cross_val_predict

In [0]:
test = pd.read_csv('test_features.csv')
train = pd.read_csv('train_features.csv')
y = pd.read_csv('train_labels.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Baseline Model

In [16]:
train.shape

(1309457, 103)

In [17]:
pd.set_option('display.max_columns', 103)
train.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,1246716,NaN,22000.0,22000.0,36 months,5.32%,662.53,A,A1,Transition Coordinator,10+ years,MORTGAGE,126000.0,NaN,NaN,home_improvement,Home improvement,604xx,IL,28.99,0.0,Oct-2000,0.0,43.0,NaN,12.0,0.0,9371.0,51%,18.0,w,0.0,29.0,Individual,NaN,NaN,0.0,0.0,367985.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18351.0,NaN,NaN,NaN,2.0,30665.0,3250.0,69.0,0.0,0.0,145.0,182.0,5.0,2.0,2.0,57.0,NaN,18.0,NaN,2.0,3.0,7.0,4.0,4.0,7.0,7.0,9.0,7.0,12.0,NaN,0.0,0.0,2.0,89.0,0.0,0.0,0.0,451899.0,198749.0,9750.0,210298.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,1678744,NaN,10000.0,10000.0,36 months,5.32%,301.15,A,A1,Director HQ Controlling,4 years,MORTGAGE,120000.0,NaN,NaN,debt_consolidation,Debt consolidation,019xx,MA,10.69,1.0,Jan-2002,0.0,13.0,NaN,13.0,0.0,9145.0,35%,46.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,40945.0,0.0,9.0,0.0,4.0,13.0,31800.0,66.0,0.0,0.0,5548.0,55.0,26100.0,0.0,1.0,0.0,4.0,3150.0,16355.0,35.9,0.0,0.0,168.0,116.0,31.0,13.0,0.0,31.0,NaN,20.0,13.0,0.0,2.0,2.0,3.0,4.0,41.0,4.0,5.0,2.0,13.0,0.0,0.0,0.0,0.0,97.8,0.0,0.0,0.0,74267.0,40945.0,25500.0,48167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,1149018,NaN,7000.0,7000.0,36 months,6.97%,216.05,A,A3,NaN,NaN,RENT,80000.0,NaN,NaN,major_purchase,Major purchase,653xx,MO,29.39,0.0,Jan-2002,2.0,NaN,NaN,13.0,0.0,3703.0,10.4%,20.0,f,0.0,NaN,Individual,NaN,NaN,0.0,0.0,78269.0,1.0,5.0,2.0,3.0,7.0,74566.0,66.0,2.0,3.0,546.0,45.0,35500.0,2.0,0.0,5.0,6.0,6021.0,17930.0,6.1,0.0,0.0,170.0,44.0,4.0,4.0,0.0,4.0,NaN,1.0,NaN,0.0,3.0,6.0,4.0,5.0,10.0,8.0,10.0,6.0,13.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,131514.0,78269.0,19100.0,96014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,1349740,NaN,1400.0,1400.0,36 months,13.67%,47.63,C,C4,Administrative Assistant,5 years,RENT,49000.0,NaN,NaN,medical,Medical expenses,770xx,TX,25.07,0.0,Aug-2004,0.0,46.0,NaN,12.0,0.0,19979.0,79.6%,18.0,w,0.0,NaN,Individual,NaN,NaN,0.0,0.0,38644.0,1.0,1.0,1.0,1.0,5.0,18665.0,96.0,0.0,2.0,7612.0,87.0,25100.0,0.0,0.0,0.0,3.0,3220.0,1411.0,93.2,0.0,0.0,111.0,136.0,16.0,5.0,0.0,16.0,NaN,16.0,46.0,0.0,7.0,8.0,7.0,8.0,2.0,11.0,16.0,8.0,12.0,0.0,0.0,0.0,1.0,94.4,100.0,0.0,0.0,44615.0,38644.0,20800.0,19515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,1234365,NaN,19200.0,19200.0,36 months,8.18%,603.26,B,B1,Plant Electrical Engineer,< 1 year,RENT,150000.0,NaN,NaN,credit_card,Credit card refinancing,971xx,OR,16.18,0.0,Oct-1997,3.0,NaN,78.0,8.0,1.0,27000.0,56%,17.0,f,0.0,31

In [26]:
train.select_dtypes(exclude=np.number).head()

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,desc,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,application_type,sec_app_earliest_cr_line,disbursement_method
0,36 months,5.32%,A,A1,Transition Coordinator,10+ years,MORTGAGE,NaN,home_improvement,Home improvement,604xx,IL,Oct-2000,51%,w,Individual,NaN,Cash
1,36 months,5.32%,A,A1,Director HQ Controlling,4 years,MORTGAGE,NaN,debt_consolidation,Debt consolidation,019xx,MA,Jan-2002,35%,w,Individual,NaN,Cash
2,36 months,6.97%,A,A3,NaN,NaN,RENT,NaN,major_purchase,Major purchase,653xx,MO,Jan-2002,10.4%,f,Individual,NaN,Cash
3,36 months,13.67%,C,C4,Administrative Assistant,5 years,RENT,NaN,medical,Medical expenses,770xx,TX,Aug-2004,79.6%,w,Individual,NaN,Cash
4,36 months,8.18%,B,B1,Plant Electrical Engineer,< 1 year,RENT,NaN,credit_card,Credit card refinancing,971xx,OR,Oct-1997,56%,f,Individual,NaN,Cash


In [0]:
train.select_dtypes(exclude=np.number).nunique()
#drop emp_title, desc, title for good
train = train.drop(columns=['emp_title', 'desc', 'title'])
#need work in second model
#int_rate: convert to float percentage
#emp_length: convert to int (year unit)
#earliest_cr_line: convert to int (year of line)
#revol_util: convert to float percentage
#sec_app_earliest_cr_line: convert 'not applicable' NaNs
baseline_train = train.drop(columns = ['int_rate', 'emp_length', 'zip_code',
                                       'earliest_cr_line', 'revol_util',
                                       'sec_app_earliest_cr_line'])

In [29]:
baseline_train.select_dtypes(exclude=np.number).nunique()

term                    2
grade                   7
sub_grade              35
home_ownership          6
purpose                14
addr_state             51
initial_list_status     2
application_type        2
disbursement_method     2
dtype: int64

In [30]:
baseline_train.isnull().sum()
#1309457 are the total rows
#drop: member_id, url
#hardly anyone has a second application or joint. I think I want
#to reduce it to 'has second app' and 'has joint'

id                                           0
member_id                              1309457
loan_amnt                                    0
funded_amnt                                  0
term                                         0
installment                                  0
grade                                        0
sub_grade                                    0
home_ownership                               0
annual_inc                                   0
url                                    1309457
purpose                                      0
addr_state                                   0
dti                                        327
delinq_2yrs                                  0
inq_last_6mths                               1
mths_since_last_delinq                  660359
mths_since_last_record                 1086702
open_acc                                     0
pub_rec                                      0
revol_bal                                    0
total_acc    

In [0]:
train = train.drop(columns=['member_id','url',
                            'sec_app_mths_since_last_major_derog',
                            'sec_app_collections_12_mths_ex_med',
                            'sec_app_chargeoff_within_12_mths',
                            'sec_app_num_rev_accts',
                            'sec_app_open_act_il',
                            'sec_app_revol_util',
                            'sec_app_open_acc',
                            'sec_app_mort_acc',
                            'sec_app_inq_last_6mths'])
#kept second credit line

In [41]:
droplist = baseline_train.columns[baseline_train.isna().any()].tolist()
baseline_train = baseline_train.drop(columns = droplist)
baseline_train.head()

,id,loan_amnt,funded_amnt,term,installment,grade,sub_grade,home_ownership,annual_inc,purpose,addr_state,delinq_2yrs,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,application_type,acc_now_delinq,delinq_amnt,disbursement_method
0,1246716,22000.0,22000.0,36 months,662.53,A,A1,MORTGAGE,126000.0,home_improvement,IL,0.0,12.0,0.0,9371.0,18.0,w,Individual,0.0,0.0,Cash
1,1678744,10000.0,10000.0,36 months,301.15,A,A1,MORTGAGE,120000.0,debt_consolidation,MA,1.0,13.0,0.0,9145.0,46.0,w,Individual,0.0,0.0,Cash
2,1149018,7000.0,7000.0,36 months,216.05,A,A3,RENT,80000.0,major_purchase,MO,0.0,13.0,0.0,3703.0,20.0,f,Individual,0.0,0.0,Cash
3,1349740,1400.0,1400.0,36 months,47.63,C,C4,RENT,49000.0,medical,TX,0.0,12.0,0.0,19979.0,18.0,w,Individual,0.0,0.0,Cash
4,1234365,19200.0,19200.0,36 months,603.26,B,B1,RENT,150000.0,credit_card,OR,0.0,8.0,1.0,27000.0,17.0,f,Individual,0.0,0.0,Cash


In [12]:
y.head()

,id,charged_off
0,1246716,0
1,1678744,0
2,1149018,1
3,1349740,0
4,1234365,0


In [13]:
y['charged_off'].value_counts()

0    1048156
1     261301
Name: charged_off, dtype: int64

In [0]:
y = y.drop(columns='id')
baseline_y = y['charged_off'] == 0

In [42]:
pipe = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    DecisionTreeClassifier(max_depth=2)
)
cross_val_score(pipe, baseline_train, baseline_y, cv=5, scoring='roc_auc', n_jobs=-1)

array([0.63994209, 0.63895898, 0.6412942 , 0.64079975, 0.63845464])

In [43]:
pipe.fit(baseline_train, baseline_y)
tree = pipe.named_steps['decisiontreeclassifier']
encoder = pipe.named_steps['onehotencoder']
feature_names = encoder.transform(baseline_train).columns
feature_names[tree.feature_importances_ > 0]
#the three most important features in this set are term and grade

Index(['term_ 36 months', 'grade_A', 'grade_B'], dtype='object')